<a href="https://colab.research.google.com/github/FlorianMimolle/MatchMarket-final/blob/master/MatchMarket_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importation de librairies :
import re
import zipfile 
import requests
import numpy as np
import pandas as pd
from google.colab import files
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics  import silhouette_score
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder

df = pd.read_csv('20220201230731-all_campaigns.csv', sep=',')

#Campagne : # Ajouter les campagnes_id au fur et à mesure que la table sera updatée
def typeCampagne(numero):
    cosmetique = [33,77]
    deco = [8,20,43,76]
    mode = [2,4,6,7,22,23,25,26,27,28,30,34,35,36,37,38,39,40,41,42,78]
    #si le numéro est dans une des listes : il renvoie le mot correspondant à la campagne
    if numero in cosmetique :
        return "Cosmetique"
    elif numero in deco:
        return "Deco"
    elif numero in mode:
        return "Mode"

#Liste des catégories :
    #Matière : 
naturel = ['coton','lin','laine','cachemire','soie','caoutchouc','alpaga','cuir','chanvre','angora','tricotine', 'suède', 
            'raffia', 'métal', 'laiton', 'konjac', 'karité', 'jute', 'huiles', 'hêtre', 'daim', 'chèvre', 'chêne', 'bouleau',
            'bois', 'aluminium', 'aluminium acier', 'acier', 'acier laqué', 'abeille', 'coco', 'huile', 'denim', 'dentelle',
            'guipure', 'huiles', 'jersey', 'textile', 'tissu', 'toile', 'tweed', 'viscose']
synthetique = ['acrylique', 'caoutchouc', 'cuir végétal','denim', 'dentelle', 'fausse fourrure', 'guipure',
            'jersey','nylon','plastique','plexi','polyamide','polyester','polyurethane','polyuréthane', 'PU', 
            'PVC', 'stratifié', 'stratifié imprimé marbre', 'synthétique', 'textile', 'tissu', 'toile', 'tweed',
            'vinyle', 'viscose', 'elasthanne']
    #Label : 
ecolabel = ['gots','oeko-tex', 'oeko tex', 'naturleder','eu ecolabel','origine france garantie', 'eco tex','eco-tex',
            'bio', 'recyclé', 'biologique', 'responsable', 'responsables', 'écoresponsable', 'éco-responsable',
            'upcyclé', 'vegan', 'vegane']
    #Couleurs:
color_vif = ['multicolor','doré','argenté','bleu algue','bleu canard','bleu lagon','bleu turquoise','jaune',
            'jaune clair','jaune pastel','jaune moutarde','jaune safran','marron terracotta','orange','rose',
            'rose framboise','rose fuchsia','rouge','rouge corail','rouge brique','rouge cerise','vert',
            'vert clair','vert menthe','vert olive','vert pistache','vert sapin','violet lavande',
            'violet lila','fleurs','mauve','brun roux',"vert d'eau",'fuschia','parme','lila','lavande',
            'moutarde','corail','safran','terrer de sienne','bandana rouge','touches multicolors',
            "menthe a   l'eau",'rose multicolore','olive','rum punch','rose 77','schiap','unicorn',
            'tie dye','rose 01','719 love bouquet','n92 rouge caftan','25 goldie red',
            'n205 - fushia irresistible - fini mat lumineux','348 coral peony - satin','cerise n107',
            'strawberry pink 02','pistache','mauve clair','lavande','jaune pastel','argenté clair','rose poudrée',
            'imprimé géométique jaune','jaune tartan','rose rouge','terracotta','rose flash','algue']
color_neutre = ['sans (transparent)','beige','beige sable','beige bouleau','beige camel','beige fauve','beige nude','beige chene','beige naturel',
            'beige hetre','blanc','blanc cassé','bleu','bleu clair','bleu ciel','bleu pastel','bleu denim','bleu foncé','bleu profond',
            'bleu klein','bleu marine','gris','gris clair','gris souris','gris foncé','gris taupe','marron',
            'marron caramel','marron cognac','marron fauve','marron noisette','marron chocolat','noir','rose clair',
            'rose saumon','rouge bordeaux','vert kaki','violet prune','fauvre','hêtre','bouleau',
            'naturel','sable','léopard','jean','nude','bleu gris','bordeaux','ecru','noir délavé',
            'camel','bleu délavé','bordeaux foncé','saumon','taupe','bandana fleur bleue','gris anthracite',
            'bleu jean','croco marron','zèbre marron','cognac','matt','14 beige copacabana','brun nude 211',
            'au naturel','marron camel','noir croco','noir semi-verni','marron croco','daim brun','brun',
            'prune','noir blanc','blanc noir','noir beige','noir bleu','bleu blanc','bois clair','lin',
            'marine']
    #Style Préférence:
style1 = ['casual', 'urbancool', 'streetwear']
style2 = ['chic', 'smart', 'working girl']
style3 = ['rock', 'gothique']
style4 = ['engagée', 'made in france']
style5 = ['fatale']
style6 = ['bohême', 'romantique']
style7 = ['vintage', 'kawaii']
style8 = ['inconnu']
    #Brand Preference:
access_brand = ["monoprix beauté"]
mass_brand = ["sephora","l'oréal","nocibé","maybelline","bourjois","marionnaud","birchbox","cosmopolitan",
            "glossier","mercu handy","bleach london"]
premium_brand = ["nuxe","benefit cosmetics","l'occitane","origins","black up","bumble & bumble","st tropez",
            "les petits prodiges","prescription lab","mimitika","claus porto"]
hdg_brand = ["urban decay","rituals","nars","opi","kerastase","essie","bobbi brown","payot","aries",
            "esthederm","bkr"]
prestige_brand = ["fenty beauty","dior backstage","kenzo","lacoste","clarins","clinique","huda beauty",
            "burberry","cacharel","make up forever","paco rabanne","marc jacobs beauty",
            "lolita lempicka","erborian","tom ford","filorga","talika","molinard"]
luxe_brand = ["chanel", "yves saint laurent", "lancome", "guerlain", "givenchy", "gucci", 
            "estée lauder", "nina ricci", "prada", "lancaster", "laneige", "sisley", "lanvin", 
            "la mer", "elizabeth arden", "acquia di parma", "la prairie", "issey miyake", 
            "juliette has a gun", "goutal", "shu uemura", "kilian"]
bio_brand = ['corine de farme','mademoiselle bio','sanoflore','cattier','havana','pur aloe',
            'les fleurs de bach','egyptian magic','absolution','decleor','melvita','dr hauschka',
            'herb essentials','abyssea','jonzac','akane','argiletz','lov organic','beliflor','leonor greyl',
            'centifolia','florame','les huilettes','patyka','finessence','bivouak','les poulettes',
            'lamazuna','pachamamai','greenma','oden','boulado','mamaki','kure bazaar','karethic']
    #Mode Preference:
access_mode = ['h&m', 'pimkie', 'pull&bear', 'asos', 'undiz', 'jennyfer', 'camaieu', 'c&a', 'kaporal', 
            'hollister', 'missguided', 'jack & jones', 'urban outfitters', 'vero moda', 'blancheporte', 
            'see u soon', 'stradivarius', 'shein', 'bershka', 'cache cache', 'boohoo', 'kiabi', 'bonobo', 
            'adopt', 'gemo', 'decathlon', 'la halle', 'bizzbee', 'la redoute', 'new look', 'monoprix']
mass_mode = ['etam', 'adidas', 'zara', 'nike', "levi's", 'adidas originals', 'converse', 'etam lingerie', 'vans',
            'zalando', 'puma', 'promod', 'naf naf', 'dr. martens', 'le coq sportif', 'fila', 'reebok', 'eastpak',
            'teddy smith', 'asics', 'le temps des cerises', 'ikks', 'les tropéziennes', 'pepe jeans', 'champion',
            'lancaster', 'quiksilver', 'lee cooper', 'kappa', 'princesse tam.tam', 'superdry', 'roxy', 'gap',
            'birkenstock', 'rip curl', 'gambettes box', 'passionata', 'under armour', 'oysho', 'lookiero', 
            "o'neill", 'scotch & soda', 'steve madden', 'oakley', 'paul hewitt', 'juicy couture', 'superga', 
            'miss selfridge', 'blugirl', 'j.crew', 'speedo', 'thecloset', 'melissa', 'jj brands', 'mango', 
            'calzedonia', 'armand thierry', 'prettylittlething', 'dim', 'desigual', 'ange', 'courir', 
            'darjeeling', 'kookai', 'lollipops', 'american eagle', 'caroll', 'esprit', 'karl marc john', 
            'maison 123', 'new balance', 'ellesse tanker', 'armor-lux', 'crocs', 'eram', 'morgan', 'topshop', 
            'geox', 'okaidi', 'abercrombie', 'circus', 'swatch', 'jonak']
premium_mode = ['calvin klein', 'lacoste', 'tommy hilfiger', 'diesel', 'michael kors', 'polo ralph lauren', 
            'zadig & voltaire', 'cacharel', 'karl lagerfeld', 'maje', 'the kooples', 'marc jacobs', 'ba&sh', 
            'sandro', 'petit bateau', 'love moschino', 'repetto', 'victoria beckham', 'pink', 'carhartt', 
            'sézane', 'porsche design', 'see by chloé', 'paul & joe', 'red valentino', 'bocage', 'dkny', 
            'gas bijoux', 'golden goose', 'le tanneur', 'claudie pierlot', 'bel air', 'liu jo', 'bazar deluxe',
            'coach', 'coccinelle', 'furla', 'le mont st michel', 'hogan', 'veja', 'les petites', 'volcom',
            'k-way', 'patagonia', 'fred perry', 'antik batik', 'j brands', "marc o'polo", 'berenice', 
            'reminiscence', 'ted baker', 'pinko', 'moon boot', 'ami paris', 'chiara ferragni', 'a.p.c.', 
            'colmar', 'stussy', 'bape', 'anne & valentin', 'la martina', "l'eclaireur", 'allsaints', 
            'bellerose', 'paige', 'kate spade', 'karen millen', 'iro', 'bally', "church's", 'asphalte',
            'polène', 'etoile', 'from future', 'amen', 'charles jourdan', 'amélie pichard', 'tory burch', 
            'rouje', 'each x other', 'be blumarine', 'axel arigato', 'herschel supply co.', 'barbour', 
            'ancient greek sandals', 'mm6', 'notify', 'anine bing', 'jim rickey', 'belstaff', 'clergerie',
            'zespa', 'kurt geiger', 'ambra maddalena', 'anya hindmarch', 'shourouk', 'house of holland',
            'majestic filatures', 'aurelie bidermann', 'hogan rebel', 'vanessa seward', 'r.s.v.p.', 
            'charli cohen', 'falke', 'by malene birger', 'patrizia pepe', 'tosca blu', 'spanx', 'pandora', 
            'timberland', 'off-white', 'balzac', 'swarovski', 'camper', 'aubade', 'ash', 'ray-ban', 'g star',
            'kiwi saint tropez', 'miss sixty', 'camperlab', 'banana moon', 'clarks', 'aigle', 'kickers', 
            'la jaquette', 'lululemon', 'ugg', 'christine laure', 'parisienne et alors', 'des petits hauts', 
            'free lance']
hdg_mode = ['longchamp', 'kenzo', 'boss', 'hugo boss', 'the north face', 'giorgio armani', 'lancel', 'off white', 
            'canada goose', 'paul smith', 'acne studios', 'tom ford', 'max mara', 'la perla', 'vanessa bruno',
            'cerruti 1881', 'moncler', 'sonia rykiel', 'escada', 'guy laroche', 'carolina herrera', 
            'agent provocateur', 'le petite robe di chiara boni', '7 for all mankind', 'alberta ferretti', 
            'plein sud', 'christopher kane', 'mcm', 'agnelle', 'akira naka', 'pyrenex', 'diane von furstenberg', 
            'alain mikli', 'agnona', 'raf simons', 'charlotte olympia', 'borsalino', 'zucca', 'barbara bui', 
            'k. jacques', 'jacomo', 'blumarine', 'paule ka', 'paul andrew', 'alexandra golovanoff', 'zac posen', 
            'botier', 'rag & bone', 'marni', 'olympia le-tan', 'jw anderson', 'fusalp', 'tom dixon', 'ambush', 
            'malo', 'harris wharf london', 'opening ceremony', 'msgm', 'sacai', 'etro', 
            'pleats please issey miyake', 'marlies dekkers', 'panoply', 'le gramme', 'tsumori chisato',
            'equipment', 'theory', 'derek lam', 'smythson', 'sartore', 'vetements', 'aalto', 'isabel marant']
prestige_mode = ['chloé', 'jean paul gaultier', 'burberry', 'christian louboutin', 'alexander mcqueen', 'versace',
            'jacquemus', 'jimmy choo', 'cavalli', 'mugler', 'dsquared2', 'stella mccartney',
            'giuseppe zanotti', 'alexander wang', 'issey miyake', 'roberto cavalli', '3.1 phillip lim',
            'mulberry', 'rochas', 'vivienne westwood', 'salvatore ferragamo', 'sergio rossi', 
            'maison margiela', 'alexa chung', "tod's", 'baldinini', 'emilio pucci', 'missoni',
            'alexandre vauthier', 'hervé leger', 'leonard', 'donna karan', 'yohji yamamoto', 'jil sander',
            'brioni', 'rick owens', 'proenza schouler', 'nicholas kirkwood', 'ann demeulemeester', 'vanrycke',
            'dice kayek']
luxe_mode = ['chanel', 'dior', 'balenciaga', 'lous vuitton', 'gucci', 'hermès', 'givenchy', 'valentino', 'fendi', 
            'prada', 'saint laurent', 'balmain', 'céline', 'bvlgari', 'miu miu', 'van cleef & arpels',
            'bottega veneta', 'elie saab', 'boucheron', 'pierre hardy', 'brunello cucinelli', 'loewe', 
            'zuhair murad', 'emanuel ungaro', 'ermenegildo zegna', 'maison ullens', 'pomellato', 'louis vuitton']
vintage_mode = ['jean paul gaultier pre-owned', 'chanel pre-owned', 'rolex pre-owned', 'john galliano pre-owned',
            'jc de castelbajac pre-owned', 'dries van noten pre-owned']
eco_responsable_mode = ['stella mccartney', 'le chapeau', 'ganni', 'ekyog', 'hopaal', 'vanda jacintho', 
            'patagonia', 'alchemist', 'atelier unes', 're/done', "n'go", 'chaussettes orphelines',
            '1083']

#Fonctions : 

    #Pour Déterminer les coordonnées lat/Long en fonction du code postale : 
def coordonnées(code_postal):
    if code_postal in coordonnées_dict :
        return [coordonnées_dict[code_postal].split(",")[0],coordonnées_dict[code_postal].split(",")[1]]
    else:
        return None
        
    #Pour Déterminer le code INSEE en fonction du code postal : 
def codeInse(code_postal):
    if str(code_postal) in codeinse_dict :
        return codeinse_dict[str(code_postal)]
    else:
        return None
    
    #Pour Déterminer la population en fonction du code 
def Population(code_insee):
    global liste 
    #Si le code INSEE est reconnu:
    if str(code_insee) in density_dict :
        #Si la population du village est suppérieur à 5000hab on met 1 (urbain)
        if density_dict[str(code_insee)] > 5000:
            return 1
        #Sinon, On met 0 (rural)
        else:
            return 0 
    #Si le code Insee est pas reconnu, c'est les arrondissements de villes, du coup, si les premiers chiffres sont 75,13 ou 69 on met urbain
    elif (str(code_insee)[:2] == "75") | (str(code_insee)[:2] == "13") | (str(code_insee)[:2] == "69"):
        return 1
    #Sinon, on met rural par defaut
    else:
        return 0
    
    #Pour determiner le juste prix (en centimes):
def Prix(price,campaign_id,product_id):
    #Si la campaign est suppérieur à 20, les prix sont corrects sauf pour deux articles
    if campaign_id >=20 :
        if str(product_id) in ["479","181"]:
            return price*100
        else:
            return price
    #Si la campaign id est inférieur à 20, on fait *100
    else:
        return price*100
    
    #Fonction pour compter les éléments d'une liste présent dans une autre liste : 
def compteur_additif(liste_df,liste_classeur):
    compteur = 0
    for i in list(liste_df): 
        if i.lower() in list(liste_classeur): #Si l'élement de la liste à tester est dans la liste_classeur alors on ajoute 1 au compteur
            compteur += 1
    return compteur

    #Fonction pour déterminer si un element est dans une liste ou non :
def compteur_unique(liste_df,liste_classeur):
    compteur = 0
    for i in liste_df:
        if i.lower() in liste_classeur: #S'il y a des élements dans la liste à tester est dans la liste_classeur alors on renvoie 1
            compteur = 1 
    return compteur

    #Créer deux fonctions pour répartir le prix dans la colonne like_price ou dislike_price en fonction de la colonne action
def dislike(action_fact,valeur_colonne):
    if action_fact == 1:
        return valeur_colonne
    else:
        return 0
def like(action_fact,valeur_colonne):
    if action_fact == 0:
        return valeur_colonne
    else:
        return 0

    #Fonction pour déterminer le like_price globale
def mean1(liste_price): #Like_price est la série du dataframe 
    somme=0
    compte=0
    moy = 0
    for i in liste_price: #Pour tous les prix de la colonne, si le prix est différence de 0, alors on ajoute à la somme puis on fait la moyenne
        if i >0:
            somme +=i
            compte+=1
    if compte > 0:
        moy=somme/compte
    return moy  

#On créé le Dictionnaire codePostale-coordonnées-code insee:
r = requests.get("https://raw.githubusercontent.com/high54/Communes-France-JSON/master/france.json")
departement = pd.json_normalize(r.json())[["Code_postal","coordonnees_gps","Code_commune_INSEE"]] #On récupère les données qui nous interesse du json
departement["Code_postal"] = departement["Code_postal"].astype(str).apply(lambda x : "0" + x if len(x) ==4 else x) #On change le type pour qu'il prenne le 0 des codes postaux à 4 chiffres
departement["Code_commune_INSEE"] = departement["Code_commune_INSEE"].astype(str).apply(lambda x : "0" + x if len(x) ==4 else x)
departement = departement.set_index("Code_postal")#On met le code postal en index
departement = departement[departement["coordonnees_gps"] != ""] #On supprime les données vides
#On transforme les json nettoyés en dictionnaire :
coordonnées_dict = departement.to_dict()["coordonnees_gps"] 
codeinse_dict = departement.to_dict()["Code_commune_INSEE"] 
r = requests.get("https://public.opendatasoft.com/explore/dataset/population-francaise-communes/download/?format=json&timezone=Europe/Berlin&lang=fr")
density = pd.json_normalize(r.json())
density = density[["fields.annee_recensement","fields.code_insee_commune","fields.population_totale"]] #On récupère les données du json qui nous interessent
density = density.sort_values(by = "fields.annee_recensement", ascending = False).drop_duplicates(subset=["fields.code_insee_commune"],keep = "first") #On garde l'année de resencement la plus récente lorsqu'il y a plusieurs années
density = density.drop(columns = ["fields.annee_recensement"]) #On supprime la colonne année de recensement
density = density.set_index("fields.code_insee_commune") #On met le code insee en index pour transformer en dictionnaire
density_dict = density.to_dict()["fields.population_totale"]

#Conversion des types de colonnes :
df["birthdate"] = pd.to_datetime(df["birthdate"])
df["zipcode"] = df["zipcode"].astype(str)
#Remplacer les valeurs NaN par Incconu
df['material'].fillna('Inconnu', inplace = True)
df['zipcode'].fillna('Inconnu', inplace = True)
df['styles preferences'].fillna('Inconnu', inplace = True)
df['brands preferences'].fillna('Inconnu', inplace = True)
df['fashion preferences'].fillna('Inconnu', inplace = True)
#Supprimer les lignes/colonnes obseletes : 
    #email de test : 
df = df[(df['email']!='e@e.com') &
        (df['email']!='aabiaad@netvariant.com') &
        (df['email']!='ea7@test.com') &
        (df['email']!='hello@praedicters.com') &
        (df['email']!='slachats@yopmail.com')]
df.dropna(subset = ['color'], inplace = True)  # Rediscuter si utile ou si remplacer par inconnu (puis ne pas mettre +1 au groupby)
df.dropna(subset = ['price'], inplace = True)  # Rediscuter si utile ou si remplacer par inconnu (puis ne pas mettre +1 au groupby)
df = df.drop(columns = ["product reference"])
#Modification des valeurs abbérentes : 
    #Zipcode : 
df['zipcode'].replace('17 Ru', '56600', inplace=True) #Après vérification, 17 Ru correspond à une personne habitant en 56600
df["zipcode"] = df["zipcode"].apply(lambda x : x.replace(".0","")) #On enleve les .0 
df['zipcode'].replace('1', '75', inplace=True) #Après vérification les personnes ayant mis 1 sont dans le 75
df["zipcode"] = df["zipcode"].apply(lambda x : "0" + x if len(x) == 4 else #Ajoute un zéro pour les codes postaux à 4 chiffres
                                                 x + "001" if x == 75 else #Attibue subjectivement le code postale 75001 à ceux qui n'ont mis que 75. (pour la suite, ils seront considérés comme urbain donc cela ne changera pas)
                                          x[0:2]+"000" if len(x) == 3 else  #Attribue subjectivement le code préfécture pour les personnes ayant mis un code à 3 chiffres
                                             x + "000" if len(x) == 2 else x) #Attribue subjectivement le code préfécture pour les personnes ayant mis un code à 2 chiffres
    #Price : 
df["price"] = np.vectorize(Prix)(df["price"], df["campaign_id"],df["product id"]) #Calcule le prix en centime (en fonction de la campaign id et du product id)
    #Matériaux remplace les mauvaises valeurs par les bonnes :
searchfor = [',', ';', 'et', '&','%'] 
df_mixte = df[df['material'].str.contains('|'.join(searchfor))] 
df_unique = df[~df['material'].str.contains('|'.join(searchfor))]
df_unique.loc[df_unique['material'].str.contains('coton',case=False),'material'] = 'Coton'
df_unique.loc[df_unique['material'].str.contains('polyest', case=False), 'material'] = 'Polyester'
df_unique.loc[df_unique['material'].str.contains('viscose', case=False),'material'] = 'Viscose'
df_unique.loc[df_unique['material'].str.contains('soie', case=False),'material'] = 'Soie'
df_unique.loc[df_unique['material'].str.contains('lin', case=False),'material'] = 'Lin'
df_unique.loc[df_unique['material'].str.contains('cuir', case=False),'material'] = 'Cuir'
df_unique.loc[df_unique['material'].str.contains('jean', case=False),'material'] = 'Jean'
df_unique.loc[df_unique['material'].str.contains('toile', case=False),'material'] = 'Toile'
df_unique.loc[df_unique['material'].str.contains('faille', case=False),'material'] = 'Maille'
df_unique.loc[df_unique['material'].str.contains('maille', case=False),'material'] = 'Maille'
df_unique.loc[df_unique['material'].str.contains('laine', case=False),'material'] = 'Laine'
df_unique.loc[df_unique['material'].str.contains('lon', case=False),'material'] = 'Nylon'
df_unique.loc[df_unique['material'].str.contains('tweed', case=False),'material'] = 'Tweed'
df_unique.loc[df_unique['material'].str.contains('acrylique', case=False),'material'] = 'Acrylique'
df_unique.loc[df_unique['material'].str.contains('bois', case=False),'material'] = 'Bois'
df_unique.loc[df_unique['material'].str.contains('hêtre', case=False),'material'] = 'Hêtre'
df_unique.loc[df_unique['material'].str.contains('pvc', case=False),'material'] = 'PVC'
df_unique.loc[df_unique['material'].str.contains('tweed', case=False),'material'] = 'Tweed'
df_unique.loc[df_unique['material'].str.contains('ra', case=False),'material'] = 'Raffia'
df = pd.concat([df_unique,df_mixte])
df['material'] = df['material'].apply(lambda x: re.sub('\W+', ' ', x))
df['material'] = df['material'].apply(lambda x: re.sub('\d+', ' ', x))
df['material'] = df['material'].apply(lambda x: x.replace('et ',''))
df['material'] = df['material'].str.split()
    #Couleur : remplace les mauvaises valeurs par les bonnes :
df['color'] = df['color'].str.replace(' &',',')
df['color'] = df['color'].str.replace(' et',',')
df['color'] = df['color'].str.lower()
df['color'] = df['color'].apply(lambda x: re.sub("\s\d+","",x))
df['color'] = df['color'].str.replace('chêne','beige chene')
df['color'] = df['color'].str.replace('noire','noir')
df['color'] = df['color'].str.replace('verte','vert')
df['color'] = df['color'].str.replace('mc','multicolor')
df['color'] = df['color'].str.replace('fuchsia','fuschia')
df['color'] = df['color'].str.replace("coral","corail")
df['color'] = df['color'].str.replace('moutarde','jaune moutarde')
df['color'] = df['color'].str.replace('touches multicolores','multicolor')
df['color'] = df['color'].str.replace('zèbre marron','marron, blanc')
df['color'] = df['color'].str.replace('zèbre','noir, blanc')
df['color'] = df['color'].str.replace('fauve','beige fauve')
df['color'] = df['color'].str.replace('bordeaux ','bordeaux')
df['color'] = df['color'].str.replace('multicolore','multicolor')
df['color'] = df['color'].str.replace('bicolore noir, écru','noir, écru')
df['color'] = df['color'].str.replace("rouge noir","rouge, noir")
df['color'] = df['color'].apply(lambda x: x.split(', '))
    #Préférences : remplace les mauvaises valeurs par les bonnes : 
df['styles preferences'] = df['styles preferences'].apply(lambda x: x.replace(' , ', ', '))
df['styles preferences'] = df['styles preferences'].apply(lambda x: x.split(', '))
df['brands preferences'] = df['brands preferences'].str.replace('Make uo Forever','Make up Forever')
df['brands preferences'] = df['brands preferences'].apply(lambda x: x.split(', '))
df['fashion preferences'] = df['fashion preferences'].apply(lambda x: x.split(', '))
df = df.reset_index()
#Colonne Age : 
df['Age'] = df['birthdate'].apply(lambda x: (pd.datetime.now().year - x.year))
df['Age'].fillna(df['Age'].median(), inplace=True)# remplacer les Nan dans la colonne Age par la moyenne
df = df[df['Age'] >= 15]#Enlever les profils ayant moins de 15 ans
#Création des colonnes supplémentaires nécessaires au Machine Learning (ou à la création de graphique) grâce aux fonctions créées :
df["Département"] = df["zipcode"].apply(lambda x : x[:2])
df["Coordonnées"] = df["zipcode"].apply(lambda x : coordonnées(x))
df["CodeInsee"] = df["zipcode"].apply(lambda x : codeInse(x))
df["Urbain"] = df["CodeInsee"].apply(lambda x : Population(x))
df['action_fact'] = df["action"].factorize()[0]
df["Naturel"] = df["material"].apply(lambda x : compteur_unique(x,naturel))
df["Synthetique"] = df["material"].apply(lambda x : compteur_unique(x,synthetique))
df["Ecolabel"] = df["material"].apply(lambda x : compteur_unique(x,ecolabel))
df["type_Campaign"] = df["campaign_id"].apply(lambda x : typeCampagne(x))
df["vif"] = df["color"].apply(lambda x : compteur_unique(x,color_vif))
df["neutre"] = df["color"].apply(lambda x : compteur_unique(x,color_neutre))
df['Casual, Urbancool, Streetwear'] = df['styles preferences'].apply(lambda x: compteur_additif(x,style1))
df['Chic, Smart, Working Girl'] = df['styles preferences'].apply(lambda x: compteur_additif(x,style2))
df['Rock, Gothique'] = df['styles preferences'].apply(lambda x: compteur_additif(x,style3))
df['Engagée, Made in France'] = df['styles preferences'].apply(lambda x: compteur_additif(x,style4))
df['Fatale'] = df['styles preferences'].apply(lambda x: compteur_additif(x,style5))
df['Bohême, Romantique'] = df['styles preferences'].apply(lambda x: compteur_additif(x,style6))
df['Vintage, Kawaii'] = df['styles preferences'].apply(lambda x: compteur_additif(x,style7))
df['Inconnu'] = df['styles preferences'].apply(lambda x: compteur_additif(x,style8))
df["access_brand"] = df["brands preferences"].apply(lambda x : compteur_additif(x,access_brand))
df["mass_brand"] = df["brands preferences"].apply(lambda x : compteur_additif(x,mass_brand))
df["premium_brand"] = df["brands preferences"].apply(lambda x : compteur_additif(x,premium_brand))
df["hdg_brand"] = df["brands preferences"].apply(lambda x : compteur_additif(x,hdg_brand))
df["prestige_brand"] = df["brands preferences"].apply(lambda x : compteur_additif(x,prestige_brand))
df["luxe_brand"] = df["brands preferences"].apply(lambda x : compteur_additif(x,luxe_brand))
df["bio_brand"] = df["brands preferences"].apply(lambda x : compteur_additif(x,bio_brand))    
df["access_mode"] = df["fashion preferences"].apply(lambda x : compteur_additif(x,access_mode))
df["mass_mode"] = df["fashion preferences"].apply(lambda x : compteur_additif(x,mass_mode))
df["premium_mode"] = df["fashion preferences"].apply(lambda x : compteur_additif(x,premium_mode))
df["hdg_mode"] = df["fashion preferences"].apply(lambda x : compteur_additif(x,hdg_mode))
df["prestige_mode"] = df["fashion preferences"].apply(lambda x : compteur_additif(x,prestige_mode))
df["luxe_mode"] = df["fashion preferences"].apply(lambda x : compteur_additif(x,luxe_mode))
df["vintage_mode"] = df["fashion preferences"].apply(lambda x : compteur_additif(x,vintage_mode))
df["eco_responsable_mode"] = df["fashion preferences"].apply(lambda x : compteur_additif(x,eco_responsable_mode))

#Remettre en ordre les colonnes
df_ML = df[["user id","Age","Urbain",
            "Casual, Urbancool, Streetwear","Chic, Smart, Working Girl","Rock, Gothique","Engagée, Made in France","Fatale","Bohême, Romantique","Vintage, Kawaii","Inconnu",
            "access_brand","mass_brand","premium_brand","hdg_brand","prestige_brand","luxe_brand","bio_brand",
            "access_mode", "mass_mode", "premium_mode", "hdg_mode", "prestige_mode", "luxe_mode", "vintage_mode", "eco_responsable_mode", 
            "action_fact",
            "type_Campaign",
            "price",
            "Naturel","Synthetique",
            "Ecolabel",
            "vif","neutre"]]
#Créer les colonnes like/dislike en fonction du prix, matière et couleur
df_ML["like_price"] = np.vectorize(like)(df_ML["action_fact"], df_ML["price"])
df_ML["dislike_price"] = np.vectorize(dislike)(df_ML["action_fact"], df_ML["price"])
df_ML["like_Naturel"] = np.vectorize(like)(df_ML["action_fact"], df_ML["Naturel"])
df_ML["dislike_Naturel"] = np.vectorize(dislike)(df_ML["action_fact"], df_ML["Naturel"])
df_ML["like_Synthetique"] = np.vectorize(like)(df_ML["action_fact"], df_ML["Synthetique"])
df_ML["dislike_Synthetique"] = np.vectorize(dislike)(df_ML["action_fact"], df_ML["Synthetique"])
df_ML["like_Ecolabel"] = np.vectorize(like)(df_ML["action_fact"], df_ML["Ecolabel"])
df_ML["dislike_Ecolabel"] = np.vectorize(dislike)(df_ML["action_fact"], df_ML["Ecolabel"])
df_ML["like_vif"] = np.vectorize(like)(df_ML["action_fact"], df_ML["vif"])
df_ML["dislike_vif"] = np.vectorize(dislike)(df_ML["action_fact"], df_ML["vif"])
df_ML["like_neutre"] = np.vectorize(like)(df_ML["action_fact"], df_ML["neutre"])
df_ML["dislike_neutre"] = np.vectorize(dislike)(df_ML["action_fact"], df_ML["neutre"])
# Supprime les colonnes d'origines
df_ML = df_ML.drop(columns = ["action_fact","price","Naturel","Synthetique","Ecolabel","vif","neutre"])
# Séparer les df en fonction de différentes types de campagnes
df_ML_Deco = df_ML[df_ML["type_Campaign"] == "Deco"].drop(columns = ["type_Campaign"])
df_ML_Cosme = df_ML[df_ML["type_Campaign"] == "Cosmetique"].drop(columns = ["type_Campaign"])
df_ML_Fashion = df_ML[df_ML["type_Campaign"] == "Mode"].drop(columns = ["type_Campaign"])

##############BOUCLE MACHINE LEARNING CLUSTERING ################################: 
nom_df = ["Fashion","Deco","Cosmetique"]
compteur = 0 #Le compteur nous permet de nommer le fichier i correctement
for df_ML_i in [df_ML_Fashion,df_ML_Deco,df_ML_Cosme]: #On clusterise chaque df_type_campaign ; 
    #On regroupe chaque lignes appartenant au même client pour avoir une ligne par client : 
    df_ML_i = df_ML_i.groupby("user id").agg({"Age" : "first", #Pour les données spécifiques au client (qui se repète sur chaque ligne de vote) : on garde la première valeurs 
                                                "Urbain" : "first",
                                                "Casual, Urbancool, Streetwear" : "first",
                                                "Chic, Smart, Working Girl" : "first",
                                                "Rock, Gothique" : "first",
                                                "Engagée, Made in France" : "first",
                                                "Fatale" : "first",
                                                "Bohême, Romantique" : "first",
                                                "Vintage, Kawaii" : "first",
                                                "Inconnu": "first",
                                                "access_brand": "first",
                                                "mass_brand": "first",
                                                "premium_brand": "first",
                                                "hdg_brand": "first",
                                                "prestige_brand":"first",
                                                "luxe_brand": "first",          
                                                "bio_brand": "first", 
                                                "access_mode": "first",
                                                "mass_mode": "first",
                                                "premium_mode": "first",
                                                "hdg_mode": "first",
                                                "prestige_mode": "first",
                                                "luxe_mode": "first",
                                                "vintage_mode": "first",
                                                "eco_responsable_mode": "first",
                                                "like_price" : mean1, #Pour les lignes de prix, on moyenne les prix suppérieur à 0
                                                "dislike_price" : mean1,
                                                "like_Naturel" : "sum", #Pour les lignes concernant les votes, on compte pour avoir le nombre de vote like/dislike du clients par critères
                                                "dislike_Naturel" : "sum",
                                                "like_Synthetique" : "sum",
                                                "dislike_Synthetique" : "sum",
                                                "like_Ecolabel" : "sum",
                                                "dislike_Ecolabel" : "sum",
                                                "like_vif" : "sum",
                                                "dislike_vif" : "sum",
                                                "like_neutre":"sum",
                                                "dislike_neutre" : "sum"})
    #On créé un copie du df_ML_i pour scaller sur cette copie (et garder en mémoire les vrais valeurs)
    df_ML_i_scale = df_ML_i.copy()
    #On scale les colonnes avec Robust Scaler (car il y a des outliers)
    rb_scaler = RobustScaler() 
    df_ML_i_scale['Age'], \
    df_ML_i_scale['Casual, Urbancool, Streetwear'], \
    df_ML_i_scale['Chic, Smart, Working Girl'], \
    df_ML_i_scale['Rock, Gothique'], \
    df_ML_i_scale['Engagée, Made in France'], \
    df_ML_i_scale['Fatale'], \
    df_ML_i_scale['Bohême, Romantique'], \
    df_ML_i_scale['Vintage, Kawaii'], \
    df_ML_i_scale['Inconnu'], \
    df_ML_i_scale['access_brand'],\
    df_ML_i_scale['mass_brand'],\
    df_ML_i_scale['premium_brand'],\
    df_ML_i_scale['hdg_brand'],\
    df_ML_i_scale['prestige_brand'],\
    df_ML_i_scale['luxe_brand'],\
    df_ML_i_scale['bio_brand'],\
    df_ML_i_scale['access_mode'],\
    df_ML_i_scale['mass_mode'],\
    df_ML_i_scale['premium_mode'],\
    df_ML_i_scale['hdg_mode'],\
    df_ML_i_scale['prestige_mode'],\
    df_ML_i_scale['luxe_mode'],\
    df_ML_i_scale['vintage_mode'],\
    df_ML_i_scale['eco_responsable_mode'],\
    df_ML_i_scale['like_price'], \
    df_ML_i_scale['dislike_price'], \
    df_ML_i_scale['like_Naturel'], \
    df_ML_i_scale['dislike_Naturel'], \
    df_ML_i_scale['like_Synthetique'], \
    df_ML_i_scale['dislike_Synthetique'], \
    df_ML_i_scale['like_Ecolabel'], \
    df_ML_i_scale['dislike_Ecolabel'], \
    df_ML_i_scale['like_vif'], \
    df_ML_i_scale['dislike_vif'], \
    df_ML_i_scale['like_neutre'], \
    df_ML_i_scale['dislike_neutre'] = rb_scaler.fit_transform(df_ML_i_scale[["Age",
                                                                               "Casual, Urbancool, Streetwear",
                                                                               "Chic, Smart, Working Girl",
                                                                               "Rock, Gothique",
                                                                               "Engagée, Made in France",
                                                                               "Fatale",
                                                                               "Bohême, Romantique",
                                                                               "Vintage, Kawaii",
                                                                               "Inconnu",
                                                                                "access_brand",
                                                                                "mass_brand",
                                                                                "premium_brand",
                                                                                "hdg_brand",
                                                                                "prestige_brand",
                                                                                "luxe_brand",          
                                                                                "bio_brand",  
                                                                               "access_mode",
                                                                               'mass_mode',
                                                                               'premium_mode',
                                                                               'hdg_mode',
                                                                               'prestige_mode',
                                                                               'luxe_mode',
                                                                               'vintage_mode',
                                                                                'eco_responsable_mode',
                                                                                "like_price",
                                                                                "dislike_price",
                                                                                "like_Naturel",
                                                                                "dislike_Naturel",
                                                                                "like_Synthetique",
                                                                                "dislike_Synthetique",
                                                                                "like_Ecolabel",
                                                                                "dislike_Ecolabel",
                                                                                "like_vif",
                                                                                "dislike_vif",
                                                                                "like_neutre",
                                                                                "dislike_neutre"]]).T
    # Principal Composant Analysis (PCA) :
    # Faire du PCA pour réduire le nombre de colonnes et enlever les corrélations avant de faire du clustering pour améliorer le modèle
    pca = PCA(n_components = 0.95) #On indique que l'on souhaite réduire le nombre de colonne mais en gardant 95% de l'information initiale
    X_pca = pca.fit_transform(df_ML_i_scale) #On scale les données
    #Methode de la silhouette pour déterminer le nombre de cluster idéal : 
    #Le nombre de cluster idéal est obtenue pour le score_silhouette maximal : 
    max_score = 0 #Au départ on fixe le score max à 0
    k_max = 0 #et donc le nombre de cluster (k) idéal est à 0 au début
    for k in range(3,10): #On teste tout les k entre 3 et 10 (si on met 2, la silhouette determinera probablement deux clusters idéales : un gros et un rassemblant les outliers.. )
        model = KMeans(n_clusters=k).fit(X_pca) #On entraine le model
        if silhouette_score(X_pca, model.labels_) > max_score: #Si le score avec le k testé à ce tour est supérieur au score_max (au premier tour il le sera forcement)
            max_score = silhouette_score(X_pca, model.labels_) #Alors le score_max prend la valeur du score obtenue à ce tour
            k_max = k #Et le k_max devient le k testé à ce tour
    # 4 cluster semblent idéaux pour faire du clustering
    model_max = KMeans(n_clusters=k_max).fit(X_pca) #On entraine le modèle avec le k_max trouvé
    y = model_max.fit_predict(X_pca) #On determine les clusters avec le modèle entrainé
    df_ML_i["cluster"] = y #On rajoute de la colonne cluster dans le dataframe avant le scale
    df_ML_i['cluster'] = df_ML_i['cluster'].apply(lambda x: x+1)#On ajoute +1 au numéro de cluster (pour plus de lisibilité, le cluster 0 devient le cluster 1)
    
    df_ML_i.to_csv(f'{nom_df[compteur]}.csv') #On génère le csv obtenu
    compteur +=1 #Ajoute 1 au compteur créé au début du machine learning pour la gestion du nom du fichier

#Créé le dataframe utilisé pour faire les graphiques
df = df[['campaign_id','action','product name','color','material','user id','zipcode','Age','Département','Coordonnées','Urbain','vif','neutre','Naturel','Synthetique','type_Campaign']]
df.to_csv('df.csv.zip', index = False, compression = "zip")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/usr/local/lib/python3.7/dist-pack